In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from Training import model, utils, dataset, train
import numpy as np
from sklearn import preprocessing
from IPython.display import HTML
from matplotlib import animation
from mpl_toolkits.mplot3d import axes3d

In [2]:
test_model_1 = model.Model500RELUBN()
# load data 1066 data 
data1 = utils.parseGainAndBWCsv('Data/BW_Gain324.csv')
data2 = utils.parseGainAndBWCsv('Data/BW_Gain322.csv')
data3 = utils.parseGainAndBWCsv('Data/BW_Gain2.0.csv')
data4 = utils.parseGainAndBWCsv('Data/BW_Gain324_1.csv')
data1 += data2
data1 += data3
data1 += data4
print(len(data1))
prune_data = utils.get_rid_of_duplicate(data1)
count, prune_data = utils.check_same_x_different_y(prune_data)
data = np.array(prune_data).astype(float)
#data = np.array(data1).astype(float)

3736
([4e-06, 1003.5], [9.299, 37500000000.0]) ([4e-06, 1003.5], [9.293, 37500000000.0])
([4e-06, 1005.5], [9.31, 37400000000.0]) ([4e-06, 1005.5], [9.304, 37500000000.0])
([4e-06, 1007.5], [9.321, 37400000000.0]) ([4e-06, 1007.5], [9.315, 37400000000.0])
([4e-06, 1019.5], [9.386, 37000000000.0]) ([4e-06, 1019.5], [9.38, 37000000000.0])
([4e-06, 1020.0], [9.388, 36900000000.0]) ([4e-06, 1020.0], [9.383, 37000000000.0])
([4.01e-06, 1019.5], [9.396, 36900000000.0]) ([4.01e-06, 1019.5], [9.391, 36900000000.0])
([4.01e-06, 1020.0], [9.399, 36900000000.0]) ([4.01e-06, 1020.0], [9.394, 36900000000.0])
([4.02e-06, 1019.5], [9.407, 36800000000.0]) ([4.02e-06, 1019.5], [9.402, 36900000000.0])
([4.02e-06, 1020.0], [9.41, 36800000000.0]) ([4.02e-06, 1020.0], [9.405, 36800000000.0])
([4.02e-06, 1008.5], [9.353, 37100000000.0]) ([4.02e-06, 1008.5], [9.343, 37200000000.0])
([4.02e-06, 1019.5], [9.413, 36800000000.0]) ([4.02e-06, 1019.5], [9.402, 36900000000.0])
([4.02e-06, 1020.0], [9.415, 368000000

In [3]:
print(data.shape)


(1649, 2, 2)


In [4]:
data = data.reshape(data.shape[0],4)
feature = data.T[0]

### OLD NORM CODE ###
# #ensure norm and denorm work 
# norm, data_min, data_max = utils.normalize(feature)
# feature_1 = utils.denormalize(norm, data_min, data_max)


# min_max = dict()
# for i in range(4):
#     feature = data.T[i]
#     norm_feature, data_min, data_max = utils.normalize(feature)
#     data.T[i] = norm_feature
#     min_max[i] = [data_min, data_max]
# norm_data, min_max = utils.normalize(data)
# denorm__data = utils.denormalize(norm_data, min_max)
### END OLD NORM CODE ###

norm_data, min_max = utils.normalize(data)
denorm_data = utils.denormalize(norm_data, min_max)
data = norm_data.reshape(norm_data.shape[0],2,2)

In [5]:
data_x = data[:, 1]
data_y = data[: ,0]
data_set = dataset.CircuitSynthesisGainAndBandwidthManually(data_x, data_y)

In [6]:
train_dataset, val_dataset = utils.splitDataset(data_set, 0.8)

In [7]:
dtype = torch.FloatTensor
loss_fn = nn.L1Loss().type(dtype)  # loss can be changed here. This is the first one i tried that gave meaningful results

x = optim.Adam
optimizer1 = x(test_model_1.parameters(), lr=3e-4)#0.06448295999961791)  # TODO haven't experimented with this yet
train_data = DataLoader(train_dataset, batch_size=250)
test_data = DataLoader(val_dataset, batch_size = 50)

In [8]:
total_train = len(train_dataset)
total_test = len(val_dataset)

In [ ]:
ta5 = []
ta10 = []
ta20 = []
va5 = []
va10 = []
va20 = []
train_loss = []
for epoch in range(2000):
    #print('Starting epoch %d / %d' % (epoch + 1, num_epochs))

    avg_loss = 0
    tcorrect5 = 0
    tcorrect10 = 0
    tcorrect20 = 0
    vcorrect5 = 0
    vcorrect10 = 0
    vcorrect20 = 0
    test_model_1.eval()
    for t, (x, y) in enumerate(test_data):

        x_var = torch.autograd.Variable(x.type(dtype))
        y_var = torch.autograd.Variable(y.type(dtype).float())

        # make predictions
        scores = test_model_1(x_var)
        

        v1_correct5 = torch.absolute(scores[:,0] - y_var[:,0]) < 0.05
        v2_correct5 = torch.absolute(scores[:,1] - y_var[:,1]) < 0.05
        vcorrect5 += torch.logical_and(v1_correct5, v2_correct5).sum().item()
        v1_correct10 = torch.absolute(scores[:,0] - y_var[:,0]) < 0.1
        v2_correct10 = torch.absolute(scores[:,1] - y_var[:,1]) < 0.1
        vcorrect10 += torch.logical_and(v1_correct10, v2_correct10).sum().item()
        v1_correct20 = torch.absolute(scores[:,0] - y_var[:,0]) < 0.2
        v2_correct20 = torch.absolute(scores[:,1] - y_var[:,1]) < 0.2
        vcorrect20 += torch.logical_and(v1_correct20, v2_correct20).sum().item()
        
    test_model_1.train()
    for t, (x, y) in enumerate(train_data):

        x_var = torch.autograd.Variable(x.type(dtype))
        y_var = torch.autograd.Variable(y.type(dtype).float())

        # make predictions
        scores = test_model_1(x_var)

        loss = loss_fn(scores.float(), y_var.float())
        avg_loss += (loss.item() - avg_loss) / (t+1)
        

        t1_correct5 = torch.absolute(scores[:,0] - y_var[:,0]) < 0.05
        t2_correct5 = torch.absolute(scores[:,1] - y_var[:,1]) < 0.05
        tcorrect5 += torch.logical_and(t1_correct5, t2_correct5).sum().item()
        t1_correct10 = torch.absolute(scores[:,0] - y_var[:,0]) < 0.1
        t2_correct10 = torch.absolute(scores[:,1] - y_var[:,1]) < 0.1
        tcorrect10 += torch.logical_and(t1_correct10, t2_correct10).sum().item()
        t1_correct20 = torch.absolute(scores[:,0] - y_var[:,0]) < 0.2
        t2_correct20 = torch.absolute(scores[:,1] - y_var[:,1]) < 0.2
        tcorrect20 += torch.logical_and(t1_correct20, t2_correct20).sum().item()
        optimizer1.zero_grad()
        loss.backward()
        optimizer1.step()
    
    print('t = %d, loss = %.4f' % (epoch + 1, avg_loss))
    
    ta5.append(tcorrect5 / total_train)
    ta10.append(tcorrect10 / total_train)
    ta20.append(tcorrect20 / total_train)
    va5.append(vcorrect5 / total_test)
    va10.append(vcorrect10 / total_test)
    va20.append(vcorrect20 / total_test)
    train_loss.append(avg_loss)
    acc, preds, idx = train.check_raw_accuracy(test_model_1, test_data, min_max, .05)


t = 1, loss = 0.3517
Got 30 / 660 partially correct (4.55 pct)
Got 0 / 330 correct (0.00 pct)
t = 2, loss = 0.1630
Got 32 / 660 partially correct (4.85 pct)
Got 0 / 330 correct (0.00 pct)
t = 3, loss = 0.1388
Got 34 / 660 partially correct (5.15 pct)
Got 0 / 330 correct (0.00 pct)
t = 4, loss = 0.1068
Got 35 / 660 partially correct (5.30 pct)
Got 0 / 330 correct (0.00 pct)
t = 5, loss = 0.0953
Got 73 / 660 partially correct (11.06 pct)
Got 0 / 330 correct (0.00 pct)
t = 6, loss = 0.0866
Got 107 / 660 partially correct (16.21 pct)
Got 0 / 330 correct (0.00 pct)
t = 7, loss = 0.0833
Got 164 / 660 partially correct (24.85 pct)
Got 0 / 330 correct (0.00 pct)
t = 8, loss = 0.0767
Got 178 / 660 partially correct (26.97 pct)
Got 9 / 330 correct (2.73 pct)
t = 9, loss = 0.0780
Got 201 / 660 partially correct (30.45 pct)
Got 26 / 330 correct (7.88 pct)
t = 10, loss = 0.0718
Got 226 / 660 partially correct (34.24 pct)
Got 54 / 330 correct (16.36 pct)
t = 11, loss = 0.0681
Got 249 / 660 partially

In [1]:
plt.figure(figsize = (20,5))
plt.plot(ta5, color='grey', label='5 percent margin')
plt.plot(ta10, color='blue', label = '10 percent margin')
plt.plot(ta20, color='red', label = '20 percent margin')
plt.legend(title = 'accuracy plot training without batch norm')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(va5, color='grey', label='5 percent margin')
plt.plot(va10, color='blue', label = '10 percent margin')
plt.plot(va20, color='red', label = '20 percent margin')
plt.legend(title = 'accuracy plot validation without batch norm')
plt.show()

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(train_loss, color='grey', label='train loss')
plt.legend(title = 'training loss without batch norm')
plt.show()

In [ ]:
max(va5)

In [ ]:
max(ta5)